In [1]:
%matplotlib widget

## Variograms Example - Ground Elevation Data for Dresden

### Import Packages

In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
from ipywidgets import *

<div align=center>
<em> <font size=4> "Everything is related to everything else, but near things are more related than distant things!"  </font size></em> 
<br>
<i> Waldo Tobler (1970) </i> </div align>

### Variograms
In order to evaluate the dependency of a variable's value on its position, all samples are compared with regard to their similarity and proximity. We will use a small data set with the ground height at 35 randomly determined positions in the city of Dresden:
### Load Data from `.csv`-file

In [3]:
# load the data from the excel-file, resulting in a pandas.DataFrame
#     (a table-like data structure)
# full path, e.g., "C:/User/.../HGHCM/Data_Dresden.csv" can be used as well
df = pd.read_csv("Data_Dresden.csv", sep=";", encoding='latin-1')

# print shape of data (n_rows, n_cols)
print(df.shape)
# print first 5 rows of data set
df.head()

(35, 5)


,Ort,Nummer,Rechtswert [m],Hochwert [m],Hoehe [m ue. NN]
0,Altmarkt,1,5411525,5656080,114
1,Zwinger,2,5411250,5656485,110
2,Hbf,3,5411185,5655012,115
3,Chemiebau,4,5410975,5653670,143
4,Wasaplatz,5,5413006,5653677,123


### Cumulative Histogram
In order to get an overview on the distribution of ground heights all over the city, we will take a look on a cumulative histogram of the values, assuming that the sum curve of the real ground heights would look similar:

In [4]:
# calculate mean and median
# with DataFrame.iloc, we can access the values in the table indexed by
#     [row, col] --> (a colon ":" can be used to say something like "from here
#     until the end" or "from the beginning until here")
mean = np.mean(df.iloc[:,-1])
median = np.median(df.iloc[:, -1])

# plot histogram
ax = df.iloc[:, -1].hist(cumulative=True, density=True, histtype="step", lw=2,
                         bins=df.shape[0], label="Ground Height", figsize=(10, 6))
# plot mean and median
ax.axvline(x=median, ls='--', color='red', label="Median")
ax.axvline(x=mean, ls='--', color='green', label="Mean")

# set axis labels, make settings
ax.set_xlabel("ground height [MASL]")
ax.set_ylabel("density")
ax.set_title("Histogram of Ground Height")
ax.set_ylim(0, 1)
ax.set_xlim(100, 300)
ax.grid(True, alpha=0.3)
l = ax.legend(loc=(0.7, 0.6))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The green line gives the arithmetic mean of all measured ground heights, the red line gives the median. As the median crosses the sum curve at approximately <span>&asymp;</span>140 m, whilst the arithmetic mean of the values is much bigger (<span>&asymp;</span>165 m), we can assume that the ground height is <b>not</b> normally distributed among the area. This indicates, that most of the <i>conventional</i> statistical measures (such as variance, standard deviation, arithmetic mean) are not meaningful, and a more sophisticated statistical evaluation is needed. However, as we are using the geographical position as additional information, the lack of a normal distribution is <b>not</b> problematic for us any more.

### Calculate Distances
In order to assess the dependency of the ground height from its position (and thus from its proximity of one known point to another), we have to create an empirical variogram. For this, we need to forget about the absolute values and positions of our "sampling points" and calculate the distances in space and the height differences between all possible pairs of points:

In [5]:
# initialize lists to store distance values in
distances = []
hdiffs = []

# iterate through all points (outter loop)
# the indexing of the table is as before
# with zip(...), we can combine multiple lists; when we then iterate over the zip,
#     we get a tuple (value0 from list0, value1 from list1, ...) at each iteration
for point1 in zip(df.iloc[:, 2], df.iloc[:, 3], df.iloc[:, 4]):
    # make lists to store information in relation to each point in the outter
    #     iteration loop
    dist_from_point1 = []
    hdiff_from_point1 = []
    # iterate through all points (inner loop)
    for point2 in zip(df.iloc[:, 2], df.iloc[:, 3], df.iloc[:, 4]):
        
        # calculate the distance in the x-direction dx
        dx = point2[0] - point1[0]
        
        # calculate the distance in the y-direction dy
        dy = point2[1] - point1[1]
        
        # calculate abolute distance
        dist = ((dx ** 2) + (dy ** 2)) ** 0.5
        
        # calculate SQUARED height difference
        hdiff = (point1[2] - point2[2]) ** 2
        
        # append results to dist_from_point1 and hdiff_from_point1
        dist_from_point1.append(dist)
        hdiff_from_point1.append(hdiff)
        
    
    # append all distances from one point to all
    # other points to the distances array
    distances.append(dist_from_point1)
    hdiffs.append(hdiff_from_point1)

### Visual Representation of Distances and Ground Height Differences
We can get a overview on the distances between the different sampling points by creating a color bar. The same can be done for the (squared) height differences between the points:

In [6]:
# set up a figure with two subplots (one row, two columns)
fig, ax = plt.subplots(ncols=2, figsize=(10, 5))

# plot representation of distances
im1 = ax[0].imshow(distances, cmap="rainbow")
# make colorbar
cbar1 = plt.colorbar(im1, shrink=0.3, ax=ax[0])
# labels etc.
cbar1.set_label("distance [m]")
ax[0].set_title("Distances Between Points")
ax[0].set_xlabel("point index")
ax[0].set_ylabel("point_index")

# plot represenation of squared height differences
im2 = ax[1].imshow(hdiffs, cmap="rainbow")
# make colorbar
cbar2 = plt.colorbar(im2, shrink=0.3, ax=ax[1])
# labels etc.
cbar2.set_label("squared height difference [m]")
ax[1].set_title("Squared Height Difference Between Points")
ax[1].set_xlabel("point index")
ax[1].set_ylabel("point_index")

# use tight layout to correct for overlapping parts of figure
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Sort Distances
In order to create an empirical variogram, we want to rank all pairs of values (i.e., distances and height differences between points), starting from the pairs with the smallest distance between each other. Ultimately, we want to have a data-structure (or table) with the rows being individual pairs of points and the columns being something like "point 1", "point 2", "distance", and "height difference".

In [7]:
# initialize list to store distances and other information
preprocessed_data = []

# iterate over upper part of data matrices, excluding the main diagonal
#     because matrix is square, symmetric and main diagonal only contains
#     zeros in both cases
for row in range(0, len(distances) - 1):
    for col in range(row + 1, len(distances)):
        
        # information = [point1, point2, dist, hdiff]
        # access distances and hdiffs by row and column indices
        information = [row, col, distances[row][col], hdiffs[row][col]]
        
        # save data
        preprocessed_data.append(information)
        
# put data to DataFrame
df_preprocessed = pd.DataFrame(data=preprocessed_data, columns=["point1", "point2",
                                                                "distance", "hdiff"])

# sort data according to distance
df_sorted = df_preprocessed.sort_values(by=["distance"])
# reindex DataFrame to start at 0 again and increase because the sorting also
#     influences the indices of the rows
df_sorted.index = [i for i in range(len(df_sorted))]

In [8]:
df_sorted.head()

,point1,point2,distance,hdiff
0,0,1,489.540601,16
1,2,29,993.227064,1
2,25,33,1090.875337,1
3,0,2,1120.813990,1
4,0,29,1182.642803,4


### Function to Calculate Empirical Variogram Values Based on Number of Classes / Number of Values in each Class
The sorted values are summarized in classes and the average distance between all values in one class is defined as the "class center". Averaging the height difference in this class allows to plot each class in the empirical variogram: 

In [9]:
def variogram(data, class_col, var_col, nvals, plotting=True):
    """
    data : pd.DataFrame containing relevant and sorted data, pd.DataFrame
    class_col: column to use to calculate class centers, int
    var_col : column to use for the variogram data, int
    nvals : number of values to consider in each class, int
    """
    
    # calculate the number of classes
    # math.ceil gets the next larger integer number
    nclasses = math.ceil(len(data) / nvals)
    
    # split array into n different classes
    data_split = np.array_split(data, nclasses)
    
    # get center of class and variogram values
    ccenters = []
    var_values = []
    
    # iterate over each class
    for c in data_split:
        # calculate class center ((min - max) / 2)
        ccenter = (c.iloc[-1, class_col] + c.iloc[0, class_col]) / 2
        # calculate variogram value
        # Note: here, the difference is NOT SQUARED because we already work with
        #     squared height differences
        var_value = (c.iloc[:, var_col].sum() / (2 * len(c)))
        # append the calculated values to the lists
        ccenters.append(ccenter)
        var_values.append(var_value)
    
    if plotting:
        #Plot the empirical variogram
        fig = plt.figure(figsize=(8,6))
        ax = fig.add_subplot(1,1,1)
        ax.scatter(ccenters, var_values, marker="x", c="black", label="empirical variogram")
        # labels etc.
        ax.set_xlabel("distance [m]")
        ax.set_ylabel("variogram $[m^2]$")
        ax.set_title("Empirical Variogram")
        ax.grid(True, alpha=0.4, zorder=0)
        
    return ccenters, var_values

In [10]:
n_values_in_class = 30
ccenters, var_values = variogram(data=df_sorted, class_col=2, var_col=3, nvals=n_values_in_class)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Visually interpreting the empirical variogram, we can see that the variance among the values that are closer than 2000 m to each other is <1000 m², which is much smaller than the absolute variance of the data set (<span>&asymp;</span>3600 m²). This indicates a strong relationship between values, which are close to each other and means that we can estimate the ground height at a certain point very well, if we know the height at other point(s) in close vicinity. After 6000 m distance, however, the variance within the classes exceeds the absolute variance, sometimes even by far. <br>
It might seem strange that some classes (=set of values with a certain distance to each other) show a much higher variance than the overall data set, but you should always keep in mind that our data is <b>not</b> normally distributed and that the choice of sampling points or spatial patterns within the data set can cause higher variations in ground height than the "average" variance.<br>
However, with this empirical variogram, we can roughly say that the data set allows us to draw good conclusions on all points that are closer than 6000 m to our sampling points. <br>
<br>
But how can we apply this knowledge for points that are not at the exact distance of our class centers? For this, we will have to find an analytical function, describing our empirical variogram best:

### Defining Functions to Calculate Residuals Between Empirical and Theoretical Variogram Values
For this, we will choose a certain set of theoretical variograms and define, how the height ($h$) depends on the distance ($d$). $\sigma^2$ is the absolute variance of the ground height, and $\lambda$, $\alpha$ and $n$ are plotting parameters.<br>
We will use four functions:
- an exponential variogram <br>
$\large h(d)=\sigma^2 \cdot (1-e^{-d/{\lambda}}) $
- a Gaussian variogram <br>
$\large h(d)=\sigma^2 \cdot (1-e^{-(d/\lambda)^2}) $
- a spherical variogram<br>
$\large h(d)=\sigma^2 \cdot \frac{h}{2\lambda} \cdot[3-(\frac{d}{\lambda})^2] $
- a power function variogram<br>
$\large h(d)=n \cdot d^{\alpha} $

In [11]:
def exp_var(x, class_centers, y_true=None, fit=True):
    """
    x : array [nugget, variance, correlation_length] of parameters, array
    class_centers : class centers / x-values to calculate variogram at, array
    y_true : observed variogram values (empirical variogram), array
    
    NOTES: this function takes class centers and observed variogram values
    and returns an array of residuals between the theoretical function and
    observed values
    """
    
    # calculate the theoretical variogram value
    exp = [(x[0] + (x[1] - x[0]) * (1 - np.exp(- i / x[2]))) for i in class_centers]
    
    if fit:
        # if fit is True, return an array of residuals (i.e., the deviations
        #     of the theoretical variogram values from the empirical variogram
        #     values)
        return np.array(np.array(exp) - np.array(y_true))
    else:
        # if fit is False, return only the theoretical variogram values at
        #     the given x-values
        return exp
        

def gauss_var(x, class_centers, y_true=None, fit=True):
    """
    x : array [nugget, variance, correlation_length] of parameters, array
    class_centers : class centers to calculate variogram at, array
    y_true : observed variogram values (empirical variogram), array
    
    NOTES: this function takes class centers and observed variogram values
    and returns an array of residuals between the theoretical function and
    observed values
    """
    
    # calculate the theoretical variogram value
    gauss = [(x[0] + (x[1] - x[0]) * (1 - np.exp(-1 * ((i / x[2]) ** 2)))) for i in class_centers]
    
    if fit:
        # if fit is True, return an array of residuals (i.e., the deviations
        #     of the theoretical variogram values from the empirical variogram
        #     values)
        return np.array(np.array(gauss) - np.array(y_true))
    else:
        # if fit is False, return only the theoretical variogram values at
        #     the given x-values
        return gauss

def spherical_var(x, class_centers, y_true=None, fit=True):
    """
    x : array [nugget, variance, correlation_length] of parameters, array
    class_centers : class centers to calculate variogram at, array
    y_true : observed variogram values (empirical variogram), array
    
    NOTES: this function takes class centers and observed variogram values
    and returns an array of residuals between the theoretical function and
    observed values
    """
    
    # calculate the theoretical variogram value
    spherical = [(x[0] + (x[1] - x[0]) * 0.5 * i / x[2] * (3 - (i / x[2]) ** 2)) if i < x[2] else x[1] for i in class_centers]
    
    if fit:
        # if fit is True, return an array of residuals (i.e., the deviations
        #     of the theoretical variogram values from the empirical variogram
        #     values)
        return np.array(np.array(spherical) - np.array(y_true))
    else:
        # if fit is False, return only the theoretical variogram values at
        #     the given x-values
        return spherical

def power_var(x, class_centers, y_true=None, fit=True):
    """
    x : array [nugget, slope, power] of parameters, array
    class_centers : class centers to calculate variogram at, array
    y_true : observed variogram values (empirical variogram), array
    
    NOTES: this function takes class centers and observed variogram values
    and returns an array of residuals between the theoretical function and
    observed values
    """
    
    # calculate the theoretical variogram value
    power = [(x[0] + (x[1] - x[0]) * i ** x[2]) for i in class_centers]
    
    if fit:
        # if fit is True, return an array of residuals (i.e., the deviations
        #     of the theoretical variogram values from the empirical variogram
        #     values)
        return np.array(np.array(power) - np.array(y_true))
    else:
        # if fit is False, return only the theoretical variogram values at
        #     the given x-values
        return power

# Manual Fitting of Theoretical Variograms
Now, we will fit the functions for our theoretical variograms to our empirical variograms manually (change the number of values in each class and change the sliders!):

In [12]:
n_values_in_class = 30

In [17]:
# get empirical variogram
ccenters, var_values = variogram(data=df_sorted, class_col=2, var_col=3, nvals=n_values_in_class, plotting=False)
xs = np.arange(0, max(ccenters), 100)

# make the base-plot
fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(ccenters, var_values, marker="x", c="black", label="empirical variogram")

line_exp, = ax.plot(xs, exp_var(x=[0., 1000., 100.], class_centers=xs, fit=False), label="Exponential", c="red")
line_gauss, = ax.plot(xs, gauss_var(x=[0., 1000., 100.], class_centers=xs, fit=False), label="Gaussian", c="deepskyblue")
line_spherical, = ax.plot(xs, spherical_var(x=[0., 1000., 100.], class_centers=xs, fit=False), label="Spherical", c="limegreen")
line_power, = ax.plot(xs, power_var(x=[0., 1., 1.], class_centers=xs, fit=False), label="Power", c="purple")

ax.set_ylim(0, max(var_values) * 1.2)
ax.set_xlim(0, max(xs) * 1.1)
ax.grid(which="both")
ax.legend(loc="best")

def update(exp_ng=0.,
           exp_vr=1000.,
           exp_cl=1000.,
           gau_ng=0.,
           gau_vr=1000.,
           gau_cl=1000.,
           sph_ng=0.,
           sph_vr=1000.,
           sph_cl=1000.,
           pwr_ng=0.,
           pwr_sp=1.,
           pwr_pw=1.):
    line_exp.set_ydata(exp_var(x=[exp_ng, exp_vr, exp_cl], class_centers=xs, fit=False))
    line_gauss.set_ydata(gauss_var(x=[gau_ng, gau_vr, gau_cl], class_centers=xs, fit=False))
    line_spherical.set_ydata(spherical_var(x=[sph_ng, sph_vr, sph_cl], class_centers=xs, fit=False))
    line_power.set_ydata(power_var(x=[pwr_ng, pwr_sp, pwr_pw], class_centers=xs, fit=False))
    fig.canvas.draw_idle()

style1 = {'description_width': '200px', "handle_color": "red"}
style2 = {'description_width': '200px', "handle_color": "deepskyblue"}
style3 = {'description_width': '200px', "handle_color": "limegreen"}
style4 = {'description_width': '200px', "handle_color": "purple"}
interact(update,
         exp_ng=widgets.FloatSlider(value=0., min=0., max=15000., step=1., description="Nugget (Exponential)", layout=widgets.Layout(width="500px"), style=style1),
         exp_vr=widgets.FloatSlider(value=1000., min=0., max=15000., step=1., description="Variance (Exponential)", layout=widgets.Layout(width="500px"), style=style1),
         exp_cl=widgets.FloatSlider(value=1000., min=0., max=15000., step=1., description="Corr. Len. (Exponential)", layout=widgets.Layout(width="500px"), style=style1),
         gau_ng=widgets.FloatSlider(value=0., min=0., max=15000., step=1., description="Nugget (Gaussian)", layout=widgets.Layout(width="500px"), style=style2),
         gau_vr=widgets.FloatSlider(value=1000., min=0., max=15000., step=1., description="Variance (Gaussian)", layout=widgets.Layout(width="500px"), style=style2),
         gau_cl=widgets.FloatSlider(value=1000., min=0., max=15000., step=1., description="Corr. Len. (Gaussian)", layout=widgets.Layout(width="500px"), style=style2),
         sph_ng=widgets.FloatSlider(value=0., min=0., max=15000., step=1., description="Nugget (Spherical)", layout=widgets.Layout(width="500px"), style=style3),
         sph_vr=widgets.FloatSlider(value=1000., min=0., max=15000., step=1., description="Variance (Spherical)", layout=widgets.Layout(width="500px"), style=style3),
         sph_cl=widgets.FloatSlider(value=1000., min=0., max=15000., step=1., description="Corr. Len. (Spherical)", layout=widgets.Layout(width="500px"), style=style3),
         pwr_ng=widgets.FloatSlider(value=0., min=0., max=15000., step=1., description="Nugget (Power)", layout=widgets.Layout(width="500px"), style=style4),
         pwr_sp=widgets.FloatSlider(value=0.2, min=0., max=100., step=.01, description="Slope (Power)", layout=widgets.Layout(width="500px"), style=style4),
         pwr_pw=widgets.FloatSlider(value=1., min=0., max=10., step=.01, description="Power (Power)", layout=widgets.Layout(width="500px"), style=style4));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='Nugget (Exponential)', layout=Layout(width='500px'),…

### Fitting and Plotting Empirical and Fitted Theoretical Variograms
Now, we will fit the functions for our theoretical variograms to our empirical variograms. In the cell below, we define a function to do this.

In [14]:
def plot_variograms(class_centers, variogram_values, manual_fitting=False,
                    exp_ng=1000.,
                    exp_vr=1000.,
                    exp_cl=1000.,
                    gau_ng=1000.,
                    gau_vr=1000.,
                    gau_cl=1000.,
                    sph_ng=1000.,
                    sph_vr=1000.,
                    sph_cl=1000.,
                    pwr_ng=0.,
                    pwr_sp=1.,
                    pwr_pw=1.):
    """
    class_centers : class centers, array
    variogram_values : variogram values, array
    manual_fitting : whether to manually fit parameters in the theoretical variograms
        or to fit with scipy.least_squares, bool
    (exp / gau / sph / pwr)_ng : nugget effect for exponential / gaussian / spherical / power variogram; floats
    (exp / gau / sph)_vr : variance for exponential / gaussian / spherical variogram; floats
    (exp / gau / sph)_cl : corr. length for exponential / gaussian / spherical variogram; floats
    pwr_sp : slope for power variogram; float
    pwr_pw : power for power variogram; float
    """
    
    # x-values for theoretical variograms
    xs = np.arange(0, max(class_centers), 100)
    
    """ exponential variogram """
    if manual_fitting:
        # change these parameters for manual fitting
        # [nugget, variance, correlation_length]
        params_exp = [exp_ng, exp_vr, exp_cl]
    else:
        # scipy least squares optimization for exp
        exp_x0 = np.array([0, 3000, 3000])
        result_exp = least_squares(
            exp_var,
            exp_x0,
            kwargs={"class_centers": class_centers, "y_true": variogram_values},
            bounds=([0, 0, 0], [1000, 10000, 10000])
        )
        params_exp = result_exp.x
        resids = result_exp.fun ** 2
        ssres = resids.sum()
        sstot = np.array([(i - np.array(var_values).mean()) ** 2 for i in var_values]).sum()
        r2_exp = 1 - (ssres / sstot)
    
    # fitted exponential variogram    
    exp_fit = [(params_exp[0] + (params_exp[1] - params_exp[0]) * (1 - np.exp(- i / params_exp[2]))) for i in xs]
    
    """ gaussian variogram """
    if manual_fitting:
        # change these parameters for manual fitting
        # [nugget, variance, correlation_length]
        params_gauss = [gau_ng, gau_vr, gau_cl]
    else:
        # scipy least squares optimization for gauss
        gauss_x0 = np.array([0, 3000, 3000])
        result_gauss = least_squares(
            gauss_var,
            gauss_x0,
            kwargs={"class_centers": class_centers, "y_true": variogram_values},
            bounds=([0, 0, 0], [1000, 10000, 10000])
        )
        params_gauss = result_gauss.x
        resids = result_gauss.fun ** 2
        ssres = resids.sum()
        sstot = np.array([(i - np.array(var_values).mean()) ** 2 for i in var_values]).sum()
        r2_gauss = 1 - (ssres / sstot)
    
    # fitted gauss variogram    
    gauss_fit = [(params_gauss[0] + (params_gauss[1] - params_gauss[0]) * (1 - np.exp(-1 * ((i / params_gauss[2]) ** 2)))) for i in xs]
    
    """ shperical variogram """
    if manual_fitting:
        # change these parameters for manual fitting
        # [nugget, variance, correlation_length]
        params_spherical = [sph_ng, sph_vr, sph_cl]
    else:
        # scipy least squares optimization for spherical
        spherical_x0 = np.array([0, 3000, 3000])
        result_spherical = least_squares(
            spherical_var,
            spherical_x0,
            kwargs={"class_centers": class_centers, "y_true": variogram_values},
            bounds=([0, 0, 0], [1000, 10000, 10000])
        )
        params_spherical = result_spherical.x
        resids = result_spherical.fun ** 2
        ssres = resids.sum()
        sstot = np.array([(i - np.array(var_values).mean()) ** 2 for i in var_values]).sum()
        r2_spherical = 1 - (ssres / sstot)
    
    # fitted spherical variogram    
    spherical_fit = [(params_spherical[0] + (params_spherical[1] - params_spherical[0]) * 0.5 * i / params_spherical[2] * \
                      (3 - (i / params_spherical[2]) ** 2)) if i < params_spherical[2] else params_spherical[1] for i in xs]
    
    """ power variogram """
    if manual_fitting:
        # change these parameters for manual fitting
        # [nugget, slope, power]
        params_power = [pwr_ng, pwr_sp, pwr_pw]
    else:
        # scipy least squares optimization for power
        power_x0 = np.array([0, 1, 1])
        result_power = least_squares(
            power_var,
            power_x0,
            kwargs={"class_centers": class_centers, "y_true": variogram_values},
            bounds=([0, 0, 0], [1000, 10000, 10000])
        )
        params_power = result_power.x
        resids = result_power.fun ** 2
        ssres = resids.sum()
        sstot = np.array([(i - np.array(var_values).mean()) ** 2 for i in var_values]).sum()
        r2_power = 1 - (ssres / sstot)
    
    # fitted power variogram    
    power_fit = [(params_power[0] + (params_power[1] - params_power[0]) * i ** params_power[2]) for i in xs]
    
    
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.scatter(class_centers, variogram_values, marker="x", c="black", label="empirical variogram")
    
    ax.plot(xs, exp_fit, label="Exponential", c="red")
    ax.plot(xs, gauss_fit, label="Gaussian", c="deepskyblue")
    ax.plot(xs, spherical_fit, label="Spherical", c="limegreen")
    ax.plot(xs, power_fit, label="Power", c="purple")
    
    # print out the fitted parameters
    ax.text(1.05, 0.8, "Exponential: \n  nugget = %1.2f \n  variance = %1.2f \n  corr. length = %1.2f \n  $R^2$ = %1.2f" \
            %(params_exp[0], params_exp[1], params_exp[2], r2_exp), transform=ax.transAxes, fontsize=12)
    
    ax.text(1.05, 0.55, "Gaussian: \n  nugget = %1.2f \n  variance = %1.2f \n  corr. length = %1.2f \n  $R^2$ = %1.2f" \
            %(params_gauss[0], params_gauss[1], params_gauss[2], r2_gauss), transform=ax.transAxes, fontsize=12)
    
    ax.text(1.05, 0.3, "Spherical: \n  nugget = %1.2f \n  variance = %1.2f \n  corr. length = %1.2f  \n  $R^2$ = %1.2f" \
            %(params_spherical[0], params_spherical[1], params_spherical[2], r2_spherical), transform=ax.transAxes, fontsize=12)
    
    ax.text(1.05, 0.05, "Power: \n  nugget = %1.2f \n  slope = %1.2f \n  power = %1.2f \n  $R^2$ = %1.2f" \
            %(params_power[0], params_power[1], params_power[2], r2_power), transform=ax.transAxes, fontsize=12)
    
    ax.legend(loc="best", fontsize=14)
    ax.set_xlabel("distance [m]", fontsize=14)
    ax.set_ylabel("variogram $[m^2]$", fontsize=14)
    ax.set_title("Empirical and Fitted Theoretical Variograms", fontsize=16)
    ax.set_ylim(0, 8500)
    ax.set_xlim(0, 20000)
    ax.grid(True, alpha=0.4, zorder=0)
    plt.tight_layout()
    plt.show()
    
    return result_exp.x, result_gauss.x, result_spherical.x, result_power.x

### Creating Empirical Variogram Values and Fit Theoretical Variograms

- try different number of values in the classes
- try different initial conditions for theoretical variograms
- try different parameter bounds for theoretical variograms

In [15]:
ccenters, var_values = variogram(data=df_sorted, class_col=2, var_col=3, nvals=100, plotting=False)
p_exp, p_gauss, p_spherical, p_power = plot_variograms(class_centers=ccenters, variogram_values=var_values, manual_fitting=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …